In [1]:

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# prompt: set tensor flow for gpu

import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow is using GPU")
  # Set memory growth to prevent TensorFlow from allocating all available GPU memory
  for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
else:
  print("TensorFlow is not using GPU")

TensorFlow is using GPU


In [5]:

csv_data ='/content/drive/MyDrive/Colab Notebooks/winrate/Final_level_data/lvls_train_simulated.csv'
data = pd.read_csv(csv_data, index_col=0)

data.head()

,r_weights,b_weights,g_weights,y_weights,moves,target.yellow.goal_0,v_weights,target.blue.goal_0,target.green.goal_0,target.red.goal_0,o_weights,target.orange.goal_0,w_weights,target.drop1.goal_0,c_weights,h_weights,target.flower.goal_0,p_weights,target.smoke.goal_0,z_weights,t_weights,target.tire.goal_0,frogs,bugs,winkies,target.munchkins.goal_0,x_weights,target.colorizer.goal_0,target.colorizeb.goal_0,target.colorizev.goal_0,target.colorizey.goal_0,target.colorizeg.goal_0,helprovers,e_weights,target.energy.goal_0,target.munchkinegg.goal_0,target.bouquet.goal_0,winrate
level,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,25,25,25,25,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.10
2,25,25,25,25,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,8.22
3,25,25,10,25,25,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,40,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,21.39
4,25,25,25,25,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15.66
5,20,20,20,20,22,0,20,0,60,0,0,0,0,0,0,0,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24.86


In [6]:
data.shape

(32781, 38)

In [7]:

from tensorflow.keras.callbacks import ModelCheckpoint

# Features and target
X = data.drop(columns=['winrate'])  # Replace with your feature columns
y = data['winrate']  # Replace with your target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [7]:
100*X_train.shape[0]//1000

2622

In [13]:

# Define the DNN model
def build_dnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dropout(0.3),  # Dropout for regularization
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',  # Mean Squared Error for regression
                  metrics=['mae'])  # Mean Absolute Error
    return model

# Build and train the model
model = build_dnn_model(X_train.shape[1])

print(model.summary())

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_{epoch:05d}.keras',  # Save model with the epoch number in the filename
    save_freq=  200 * X_train.shape[0] // 1000,  # Calculate the batch frequency for every 10,000 epochs
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=2000,  # Adjust based on convergence
    batch_size=512,
    callbacks=[ checkpoint_callback ], #early_stopping
    verbose=1
)


# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 512)                 │          19,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 203,905 (796.50 KB)

 Trainable params: 202,113 (789.50 KB)

 Non-trainable params: 1,792 (7.00 KB)

None
Epoch 1/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 11s 135ms/step - loss: 571.2164 - mae: 19.9837 - val_loss: 572.1665 - val_mae: 19.8988
Epoch 2/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 531.5872 - mae: 19.3090 - val_loss: 556.6860 - val_mae: 19.5660
Epoch 3/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 474.5533 - mae: 18.0292 - val_loss: 508.1003 - val_mae: 18.3668
Epoch 4/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 396.1513 - mae: 15.9840 - val_loss: 422.4655 - val_mae: 15.9093
Epoch 5/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 308.8899 - mae: 13.2923 - val_loss: 325.2928 - val_mae: 12.8065
Epoch 6/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 232.0502 - mae: 10.6760 - val_loss: 252.1587 - val_mae: 10.2735
Epoch 7/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 187.7732 - mae: 9.0276 - val_loss: 201.2141 - val_mae: 8.7276
Epoch 8/2000
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 158.5728 - mae: 8.2338 - val_loss: 170.3306 - val_mae: 8.0279
Epoc

In [14]:
model.save('/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_dnn5_55_01.keras')

In [18]:
import glob

In [17]:
modelpaths = glob.glob('/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/*.keras')

for modelpath in modelpaths:
  print(modelpath )
  model = tf.keras.models.load_model(modelpath)

    # Evaluate the model
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f"Mean Squared Error: {mse}")
  print(f"R^2 Score: {r2}")
  print('-------------------' )

/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00128.keras
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Mean Squared Error: 71.63608760262827
R^2 Score: 0.5903228267168781
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00256.keras
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Mean Squared Error: 58.204844467622294
R^2 Score: 0.6671343040793813
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00384.keras
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Squared Error: 55.21887809603621
R^2 Score: 0.684210645118768
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00512.keras
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Squared Error: 53.56446149679409
R^2 Score: 0.6936720316697738
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00640.keras
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Squared Error

In [3]:

csv_data ='/content/drive/MyDrive/Colab Notebooks/winrate/Final_level_data/lvls_test_simulated.csv'
test_data = pd.read_csv(csv_data, index_col=0)

test_data.head()

,r_weights,b_weights,g_weights,y_weights,moves,target.yellow.goal_0,v_weights,target.blue.goal_0,target.green.goal_0,target.red.goal_0,o_weights,target.orange.goal_0,w_weights,target.drop1.goal_0,c_weights,h_weights,target.flower.goal_0,p_weights,target.smoke.goal_0,z_weights,t_weights,target.tire.goal_0,frogs,bugs,winkies,target.munchkins.goal_0,x_weights,target.colorizer.goal_0,target.colorizeb.goal_0,target.colorizev.goal_0,target.colorizey.goal_0,target.colorizeg.goal_0,helprovers,e_weights,target.energy.goal_0,target.munchkinegg.goal_0,target.bouquet.goal_0,winrate
level,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4691,20,20,10,20,24,0,0,0,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,18,15,0,0,0,0,0,0,0,0,0,0,0,13.78
3142,22,22,22,22,27,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,20.44
3266,22,22,22,22,31,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12.45
26,25,25,25,25,14,0,0,0,20,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,94.49
1214,20,20,20,20,10,0,20,65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20.84


In [4]:
test_data.shape

(1171, 38)

In [13]:
# Features and target
X_test = test_data.drop(columns=['winrate'])  # Replace with your feature columns
y_test = test_data['winrate']  # Replace with your target column


# # Normalize the features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [14]:
modelpath = '/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_dnn5_55_01.keras'


model = tf.keras.models.load_model(modelpath)

In [15]:
# Evaluate the model
y_pred = model.predict(X_test_sc)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Error: 199.11153646347427
R^2 Score: -0.12964490922095195


In [16]:
test_df=pd.DataFrame()
test_df['y_test']=y_test
test_df['y_pred']=y_pred

test_df

,y_test,y_pred
level,,
4691,13.78,22.826397
3142,20.44,15.861020
3266,12.45,16.264956
26,94.49,22.384565
1214,20.84,15.188178
...,...,...
5188,19.82,21.379139
3772,38.38,90.469406
3777,24.15,21.012487


In [20]:
modelpaths = glob.glob('/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/*.keras')

for modelpath in modelpaths:
  print(modelpath )
  model = tf.keras.models.load_model(modelpath)

    # Evaluate the model
  y_pred = model.predict(X_test_sc)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f"Mean Squared Error: {mse}")
  print(f"R^2 Score: {r2}")
  print('-------------------' )

/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00128.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Error: 178.02070816912195
R^2 Score: -0.00998761945693638
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00256.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Mean Squared Error: 183.0902294478711
R^2 Score: -0.038749181978336544
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00384.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Mean Squared Error: 192.05229598791442
R^2 Score: -0.0895948186645692
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00512.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Mean Squared Error: 199.7141244052167
R^2 Score: -0.13306364834997342
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_2/model_epoch_00640.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Err

In [21]:
modelpaths = glob.glob('/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_1/*.keras')

for modelpath in modelpaths:
  print(modelpath )
  model = tf.keras.models.load_model(modelpath)

    # Evaluate the model
  y_pred = model.predict(X_test_sc)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f"Mean Squared Error: {mse}")
  print(f"R^2 Score: {r2}")
  print('-------------------' )

/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_1/model_epoch_00128.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Mean Squared Error: 179.03747127030368
R^2 Score: -0.0157561514141209
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_1/model_epoch_00256.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Error: 185.24600147651117
R^2 Score: -0.050979798751468186
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_1/model_epoch_00384.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Error: 182.5759210923981
R^2 Score: -0.03583129070066837
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_1/model_epoch_00512.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Error: 184.0867675643516
R^2 Score: -0.04440296894679441
-------------------
/content/drive/MyDrive/Colab Notebooks/winrate/model/DNN_5_1/model_epoch_00640.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Mean Squared Erro